In [1]:
import time
import os

In [7]:
from dotenv import load_dotenv
from langchain_openai.chat_models import ChatOpenAI
import os

load_dotenv()

OPENAI_API_KEY = os.getenv('OPENAI_API_KEY')
PINECONE_API_KEY = os.getenv('PINECONE_API_KEY')

os.environ["OPENAI_API_KEY"] = OPENAI_API_KEY

llm = ChatOpenAI(model="gpt-3.5-turbo")

In [12]:
from pinecone import Pinecone, ServerlessSpec
from langchain_pinecone import PineconeVectorStore
from langchain_openai import OpenAIEmbeddings

pc = Pinecone(api_key=PINECONE_API_KEY)

index_name = "qb-rag-index"  # change if desired

existing_indexes = [index_info["name"] for index_info in pc.list_indexes()]

if index_name not in existing_indexes:
    pc.create_index(
        name=index_name,
        dimension=1536,
        metric="cosine",
        spec=ServerlessSpec(cloud="aws", region="us-east-1"),
    )
    while not pc.describe_index(index_name).status["ready"]:
        time.sleep(1)

index = pc.Index(index_name)
vector_store = PineconeVectorStore(index=index, embedding=OpenAIEmbeddings(model="text-embedding-3-small"))

In [13]:
def read_chunks_to_list(file_path):
    with open(file_path, "r") as file:
        file_content = file.read().strip()
        chunks = file_content.split("\n\n")
    return chunks


file_path = "text.txt"
chunks = read_chunks_to_list(file_path)

In [14]:
from QB_RAG.converter import Converter

In [15]:
cvt = Converter(vector_store, llm)

In [16]:
for i in chunks:
    cvt.add_documents(i)